In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import psycopg2
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from datetime import datetime

# Database Connection
def connect_db():
    return psycopg2.connect(
        host="localhost",
        database="screen_time_db",
        user="postgres",
        password="root"
    )

# Setup Database
def setup_database():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS screen_time (
            Date DATE PRIMARY KEY,
            Screen_Time FLOAT,
            Phone_Unlocks INT,
            WhatsApp INT,
            Instagram INT,
            Facebook INT,
            YouTube INT,
            Chrome INT,
            Games INT,
            Others INT
        )
    """)
    conn.commit()
    conn.close()

# Preload Data from the PDF
def preload_pdf_data():
    existing_data = [
        ("2024-09-25", 4.94, 14, 1, 0, 2, 0, 7, 6, 8),
        ("2024-09-26", 6.07, 13, 4, 2, 5, 9, 2, 6, 10),
        ("2024-09-27", 2.65, 8, 4, 3, 8, 10, 2, 0, 0),
        ("2024-09-28", 4.71, 8, 1, 8, 7, 5, 9, 10, 10),
        ("2024-09-29", 3.13, 16, 1, 1, 6, 7, 0, 10, 7),
    ]

    conn = connect_db()
    cursor = conn.cursor()

    for record in existing_data:
        try:
            cursor.execute("""
                INSERT INTO screen_time 
                (Date, Screen_Time, Phone_Unlocks, WhatsApp, Instagram, Facebook, YouTube, Chrome, Games, Others)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, record)
        except psycopg2.errors.UniqueViolation:
            pass  # Ignore duplicate entries

    conn.commit()
    conn.close()

# Save Data
def save_data():
    try:
        date = datetime.strptime(entry_date.get(), "%Y-%m-%d").date()
        screen_time = float(entry_screen_time.get())
        phone_unlocks = int(entry_phone_unlocks.get())
        whatsapp = int(entry_whatsapp.get())
        instagram = int(entry_instagram.get())
        facebook = int(entry_facebook.get())
        youtube = int(entry_youtube.get())
        chrome = int(entry_chrome.get())
        games = int(entry_games.get())
        others = int(entry_others.get())

        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO screen_time 
            (Date, Screen_Time, Phone_Unlocks, WhatsApp, Instagram, Facebook, YouTube, Chrome, Games, Others)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (date, screen_time, phone_unlocks, whatsapp, instagram, facebook, youtube, chrome, games, others))
        conn.commit()
        conn.close()

        messagebox.showinfo("Success", "Data saved successfully!")
        clear_form()
        load_data()
    except Exception as e:
        messagebox.showerror("Error", f"Failed to save data: {e}")

# Load Data
def load_data():
    for item in tree.get_children():
        tree.delete(item)

    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM screen_time ORDER BY Date DESC")
    rows = cursor.fetchall()
    conn.close()

    for row in rows:
        tree.insert("", tk.END, values=row)

# Clear Form
def clear_form():
    entry_date.delete(0, tk.END)
    entry_screen_time.delete(0, tk.END)
    entry_phone_unlocks.delete(0, tk.END)
    entry_whatsapp.delete(0, tk.END)
    entry_instagram.delete(0, tk.END)
    entry_facebook.delete(0, tk.END)
    entry_youtube.delete(0, tk.END)
    entry_chrome.delete(0, tk.END)
    entry_games.delete(0, tk.END)
    entry_others.delete(0, tk.END)

# Analyze Data and Show in GUI
def analyze_data():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT Date, Screen_Time FROM screen_time ORDER BY Date")
    data = cursor.fetchall()
    conn.close()

    if not data:
        messagebox.showinfo("No Data", "No data available for analysis.")
        return

    dates = [row[0] for row in data]
    screen_time = [row[1] for row in data]

    # Plotting
    figure = plt.Figure(figsize=(8, 5), dpi=100)
    ax = figure.add_subplot(111)
    ax.plot(dates, screen_time, marker="o", linestyle="-", color="blue", label="Screen Time (hrs)")
    ax.set_title("Daily Screen Time Analysis")
    ax.set_xlabel("Date")
    ax.set_ylabel("Screen Time (hrs)")
    ax.legend()
    ax.grid(True)

    # Embed the plot into the GUI
    for widget in frame_graph.winfo_children():
        widget.destroy()  # Clear previous graph

    canvas = FigureCanvasTkAgg(figure, frame_graph)
    canvas.get_tk_widget().pack()
    canvas.draw()

# Setup Database and Preload Data
setup_database()
preload_pdf_data()

# GUI Setup
root = tk.Tk()
root.title("Screen Time Tracker")
root.geometry("950x700")
root.configure(bg="lightgray")

# Title
tk.Label(root, text="Screen Time Tracker", font=("Arial", 20, "bold"), bg="lightgray", fg="blue").pack(pady=10)

# Form
frame_form = tk.Frame(root, bg="lightgray")
frame_form.pack(pady=10)

labels = [
    "Date (YYYY-MM-DD):", "Screen Time (hrs):", "Phone Unlocks:",
    "WhatsApp Usage:", "Instagram Usage:", "Facebook Usage:",
    "YouTube Usage:", "Chrome Usage:", "Games Usage:", "Others Usage:"
]

entries = []

for i, label_text in enumerate(labels):
    tk.Label(frame_form, text=label_text, font=("Arial", 12), bg="lightgray").grid(row=i, column=0, sticky="w", padx=10, pady=5)
    entry = tk.Entry(frame_form, width=30)
    entry.grid(row=i, column=1, padx=10, pady=5)
    entries.append(entry)

(entry_date, entry_screen_time, entry_phone_unlocks,
 entry_whatsapp, entry_instagram, entry_facebook,
 entry_youtube, entry_chrome, entry_games, entry_others) = entries

# Buttons
frame_buttons = tk.Frame(root, bg="lightgray")
frame_buttons.pack(pady=10)

tk.Button(frame_buttons, text="Save Data", command=save_data, bg="green", fg="white", font=("Arial", 12)).pack(side="left", padx=10)
tk.Button(frame_buttons, text="Analyze Data", command=analyze_data, bg="orange", fg="white", font=("Arial", 12)).pack(side="left", padx=10)

# Data Table
frame_table = tk.Frame(root, bg="lightgray")
frame_table.pack(pady=10)

columns = ["Date", "Screen Time", "Phone Unlocks", "WhatsApp", "Instagram", "Facebook", "YouTube", "Chrome", "Games", "Others"]
tree = ttk.Treeview(frame_table, columns=columns, show="headings", height=10)

for col in columns:
    tree.heading(col, text=col)
    tree.column(col, width=90)

tree.pack(pady=20)

# Graph Frame
frame_graph = tk.Frame(root, bg="lightgray")
frame_graph.pack(pady=10)

# Load Data
load_data()

root.mainloop()


OperationalError: connection to server at "localhost" (::1), port 5432 failed: FATAL:  database "screen_time_db" does not exist
